In [1]:
from __future__ import print_function

import sys
import numpy as np
from time import time
import matplotlib.pyplot as plt 

sys.path.append('/home/xilinx')
from pynq import Overlay
from pynq import allocate
from pynq import MMIO

ROM_SIZE = 0x2000 #8K

SOC_UP = 0x0000;
SOC_LA = 0x1000;
PL_AA_MB = 0x2000;
PL_AA = 0x2100;
SOC_IS = 0x3000;
SOC_AS = 0x4000;
SOC_CC = 0x5000;
PL_AS = 0x6000;
PL_IS = 0x7000;
PL_DMA = 0x8000;

In [2]:
import os
os.getcwd()

'/home/xilinx/jupyter_notebooks/ASoC_Lab_S2/Lab4'

In [3]:
ol = Overlay("/home/xilinx/jupyter_notebooks/ASoC_Lab_S2/Lab4/caravel_fpga.bit")
#ol.ip_dict

In [4]:
ipOUTPIN = ol.output_pin_0
ipPS = ol.caravel_ps_0
# ipReadROMCODE = ol.read_romcode_0

#Add for SPI
ip_QSPI = ol.axi_quad_spi_0

In [5]:
# ============================================
# AXI QuadSPI Control
# ============================================
XSP_DGIER_OFFSET = 0x1C
XSP_IISR_OFFSET = 0x20
XSP_IIER_OFFSET = 0x28
XSP_SRR_OFFSET = 0x40
XSP_CR_OFFSET = 0x60
XSP_SR_OFFSET = 0x64
XSP_DTR_OFFSET = 0x68
XSP_DRR_OFFSET = 0x6C
XSP_SSR_OFFSET = 0x70
XSP_TFO_OFFSET = 0x74
XSP_RFO_OFFSET = 0x78
XSP_REGISTERS = [0x40, 0x60, 0x64, 0x68, 0x6c, 0x70, 0x74, 0x78, 0x1c, 0x20, 0x28]

XSP_SRR_RESET_MASK = 0x0A
XSP_SR_TX_EMPTY_MASK = 0x04
XSP_SR_TX_FULL_MASK = 0x08
XSP_CR_TRANS_INHIBIT_MASK = 0x100
XSP_CR_LOOPBACK_MASK = 0x01
XSP_CR_ENABLE_MASK = 0x02
XSP_CR_MASTER_MODE_MASK = 0x04
XSP_CR_CLK_POLARITY_MASK = 0x08
XSP_CR_CLK_PHASE_MASK = 0x10
XSP_CR_TXFIFO_RESET_MASK = 0x20
XSP_CR_RXFIFO_RESET_MASK = 0x40
XSP_CR_MANUAL_SS_MASK = 0x80

SLAVE_NO_SELECTION = 0xFFFFFFFF

def cnfg(AxiQspi, clk_phase=0, clk_pol=0):
    print("Configure device")
    # Reset the SPI device
    AxiQspi.write(XSP_SRR_OFFSET, XSP_SRR_RESET_MASK)
    # Enable the transmit empty interrupt, which we use to determine progress on the transmission. 
    AxiQspi.write(XSP_IIER_OFFSET, XSP_SR_TX_EMPTY_MASK)
    # Disable the global IPIF interrupt
    AxiQspi.write(XSP_DGIER_OFFSET, 0)
    # Deselect the slave on the SPI bus
    AxiQspi.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)
    # Disable the transmitter, enable Manual Slave Select Assertion, put SPI controller into master mode, and enable it
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg | XSP_CR_MASTER_MODE_MASK | XSP_CR_MANUAL_SS_MASK | XSP_CR_ENABLE_MASK | XSP_CR_TXFIFO_RESET_MASK | XSP_CR_RXFIFO_RESET_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg & ~(XSP_CR_CLK_PHASE_MASK | XSP_CR_CLK_POLARITY_MASK) 
    if clk_phase == 1:
        ControlReg = ControlReg | XSP_CR_CLK_PHASE_MASK
    if clk_pol == 1:
        ControlReg = ControlReg | XSP_CR_CLK_POLARITY_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)

    return 0

def write_tx_fifo(AxiQspi):
    #print("TransferData")
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg & ~XSP_CR_TRANS_INHIBIT_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)

    StatusReg = AxiQspi.read(XSP_SR_OFFSET)
    while (StatusReg & XSP_SR_TX_EMPTY_MASK) == 0:
        StatusReg = AxiQspi.read(XSP_SR_OFFSET)

    #print('XSP_RFO_OFFSET  : 0x{0:08x}'.format(AxiQspi.read(XSP_RFO_OFFSET)))
    ControlReg = AxiQspi.read(XSP_CR_OFFSET)
    ControlReg = ControlReg | XSP_CR_TRANS_INHIBIT_MASK
    AxiQspi.write(XSP_CR_OFFSET, ControlReg)


def read_rx_fifo(bypass_length, AxiQspi):
    #print("ReadResponse")
    resp = list()
    RxFifoStatus = AxiQspi.read(XSP_SR_OFFSET) & 0x01
    
    # By pass the FIFO data during master issue command and address to slave device
    command_addr_length = bypass_length
    counter = 0    
    
    while RxFifoStatus == 0:
        #temp = AxiQspi.read(XSP_RFO_OFFSET)
        #print('XSP_RFO_OFFSET  : 0x{0:08x}'.format(temp))
        temp = AxiQspi.read(XSP_DRR_OFFSET)
        #print('XSP_DRR_OFFSET  : 0x{0:08x}'.format(temp))    

        counter = counter + 1
        if(counter > command_addr_length):
            resp.append(temp)        
        
        RxFifoStatus = AxiQspi.read(XSP_SR_OFFSET) & 0x01

    return resp

In [6]:
# Check MPRJ_IO input/out/en
# 0x10 : Data signal of ps_mprj_in
#        bit 31~0 - ps_mprj_in[31:0] (Read/Write)
# 0x14 : Data signal of ps_mprj_in
#        bit 5~0 - ps_mprj_in[37:32] (Read/Write)
#        others  - reserved
# 0x1c : Data signal of ps_mprj_out
#        bit 31~0 - ps_mprj_out[31:0] (Read)
# 0x20 : Data signal of ps_mprj_out
#        bit 5~0 - ps_mprj_out[37:32] (Read)
#        others  - reserved
# 0x34 : Data signal of ps_mprj_en
#        bit 31~0 - ps_mprj_en[31:0] (Read)
# 0x38 : Data signal of ps_mprj_en
#        bit 5~0 - ps_mprj_en[37:32] (Read)
#        others  - reserved

print ("0x10 = ", hex(ipPS.read(0x10)))
print ("0x14 = ", hex(ipPS.read(0x14)))
print ("0x1c = ", hex(ipPS.read(0x1c)))
print ("0x20 = ", hex(ipPS.read(0x20)))
print ("0x34 = ", hex(ipPS.read(0x34)))
print ("0x38 = ", hex(ipPS.read(0x38)))


0x10 =  0x0
0x14 =  0x0
0x1c =  0x0
0x20 =  0x0
0x34 =  0xfffffff7
0x38 =  0x3f


In [7]:
# ============================================
# Release Reset First before passthrough mode
# ============================================
# Release Caravel reset
# 0x10 : Data signal of outpin_ctrl
#        bit 0  - outpin_ctrl[0] (Read/Write)
#        others - reserved
print (ipOUTPIN.read(0x10))
ipOUTPIN.write(0x10, 1)
print (ipOUTPIN.read(0x10))

0
1


In [8]:
# ============================================
# Load firmware (fsic.hex) to memory npROM
# ============================================

# Create np with 8K/4 (4 bytes per index) size and be initiled to 0
npROM = np.zeros(ROM_SIZE >> 2, dtype=np.uint32)

npROM_index = 0
npROM_offset = 0
fiROM = open("/home/xilinx/jupyter_notebooks/ASoC_Lab_S2/Lab4/fsic.hex", "r+")

for line in fiROM:
    # offset header
    if line.startswith('@'):
        # Ignore first char @
        npROM_offset = int(line[1:].strip(b'\x00'.decode()), base = 16)
        npROM_offset = npROM_offset >> 2 # 4byte per offset
        #print (npROM_offset)
        npROM_index = 0
        continue
    #print (line)

    # We suppose the data must be 32bit alignment
    buffer = 0
    bytecount = 0
    for line_byte in line.strip(b'\x00'.decode()).split():
        buffer += int(line_byte, base = 16) << (8 * bytecount)
        bytecount += 1
        # Collect 4 bytes, write to npROM
        if(bytecount == 4):
            npROM[npROM_offset + npROM_index] = buffer
            # Clear buffer and bytecount
            buffer = 0
            bytecount = 0
            npROM_index += 1
            #print (npROM_index)
            continue
    # Fill rest data if not alignment 4 bytes
    if (bytecount != 0):
        npROM[npROM_offset + npROM_index] = buffer
        npROM_index += 1
    
fiROM.close()

In [9]:
# ============================================
# Enabling passthrou mode
# ============================================
cnfg(ip_QSPI)
# Passthrou mode - Write command
ip_QSPI.write(XSP_DTR_OFFSET, 0xC4) # Pass-Through (management)
ip_QSPI.write(XSP_DTR_OFFSET, 0x02) # Command: Write data to memory
ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Address_byte0
ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Address_byte1
ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Address_byte2

print('XSP_TFO_OFFSET  : 0x{0:08x}'.format(ip_QSPI.read(XSP_TFO_OFFSET)))

ip_QSPI.write(XSP_SSR_OFFSET, 0xFFFFFFFE)
write_tx_fifo(ip_QSPI)

print('XSP_TFO_OFFSET  : 0x{0:08x}'.format(ip_QSPI.read(XSP_TFO_OFFSET)))

Configure device
XSP_TFO_OFFSET  : 0x00000004
XSP_TFO_OFFSET  : 0x00000000


In [10]:
# ============================================
# Writing FW into SPIROM
# ============================================
# Fill up Tx_FIFO (16) for each write_tx_fifo
for index in range (ROM_SIZE >> 2):
     # 4 bytes alignment in npROM
    for byte_shift in range(4):
        tmp = int((npROM[index] >> (byte_shift * 8)) & 0xFF)
        ip_QSPI.write(XSP_DTR_OFFSET, tmp) # Write_data
    # TX_FIFO = 16, 4 * 4 = 16
    if((index % 3) == 3):
        write_tx_fifo(ip_QSPI)
        
# If rest data is not enough 16 bytes. Tx_FIFO is not empty
    StatusReg = ip_QSPI.read(XSP_SR_OFFSET)
    if ((StatusReg & XSP_SR_TX_EMPTY_MASK) == 0):
         write_tx_fifo(ip_QSPI)

In [11]:
# ============================================
# Read SPIROM for testing
# ============================================
cnfg(ip_QSPI)

Configure device


0

In [12]:
# Test Passthrou mode - Read command
ip_QSPI.write(XSP_DTR_OFFSET, 0xC4) # Pass-Through (management)
ip_QSPI.write(XSP_DTR_OFFSET, 0x03) # Command: Read data from memory
ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Address_byte0
ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Address_byte1
ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Address_byte2
# Write dummy data
data_length = 0x8
for index in range(data_length):
    ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Dummy data

print('XSP_TFO_OFFSET  : 0x{0:08x}'.format(ip_QSPI.read(XSP_TFO_OFFSET)))
ip_QSPI.write(XSP_SSR_OFFSET, 0xFFFFFFFE)

XSP_TFO_OFFSET  : 0x0000000c


In [13]:
# Issue SPI master cycle
write_tx_fifo(ip_QSPI)

# Read the Rx data
rx_final = read_rx_fifo(5, ip_QSPI)
for data in rx_final:
    print (hex(data))

0x6f
0x0
0x0
0xb
0x13
0x0
0x0
0x0


In [14]:
# Write dummy data
data_length = 0x8
for index in range(data_length):
    ip_QSPI.write(XSP_DTR_OFFSET, 0x00) # Dummy data

print('XSP_TFO_OFFSET  : 0x{0:08x}'.format(ip_QSPI.read(XSP_TFO_OFFSET)))

XSP_TFO_OFFSET  : 0x00000007


In [15]:
# Issue SPI master cycle
write_tx_fifo(ip_QSPI)

# Read the Rx data
rx_final = read_rx_fifo(0, ip_QSPI)
for data in rx_final:
    print (hex(data))

0x13
0x0
0x0
0x0
0x13
0x0
0x0
0x0


In [16]:
# ============================================
# Exit passthrou mode, FW will be fetched
# ============================================
ip_QSPI.write(XSP_SSR_OFFSET, SLAVE_NO_SELECTION)

In [17]:
# Check MPRJ_IO input/out/en
# 0x10 : Data signal of ps_mprj_in
#        bit 31~0 - ps_mprj_in[31:0] (Read/Write)
# 0x14 : Data signal of ps_mprj_in
#        bit 5~0 - ps_mprj_in[37:32] (Read/Write)
#        others  - reserved
# 0x1c : Data signal of ps_mprj_out
#        bit 31~0 - ps_mprj_out[31:0] (Read)
# 0x20 : Data signal of ps_mprj_out
#        bit 5~0 - ps_mprj_out[37:32] (Read)
#        others  - reserved
# 0x34 : Data signal of ps_mprj_en
#        bit 31~0 - ps_mprj_en[31:0] (Read)
# 0x38 : Data signal of ps_mprj_en
#        bit 5~0 - ps_mprj_en[37:32] (Read)
#        others  - reserved

print ("0x10 = ", hex(ipPS.read(0x10)))
print ("0x14 = ", hex(ipPS.read(0x14)))
print ("0x1c = ", hex(ipPS.read(0x1c)))
print ("0x20 = ", hex(ipPS.read(0x20)))
print ("0x34 = ", hex(ipPS.read(0x34)))
print ("0x38 = ", hex(ipPS.read(0x38)))

0x10 =  0x0
0x14 =  0x0
0x1c =  0x0
0x20 =  0x0
0x34 =  0x3ffff6
0x38 =  0x10


In [18]:
# ====================================================================================== #
# ====================================================================================== #
# PL_FSIC Side Configuration
# ====================================================================================== #
# ====================================================================================== #

In [19]:
# ====================================================================================== #
# PL side Configuration                                                                  #
# ====================================================================================== #

In [20]:
IP_BASE_ADDRESS = 0x60000000
ADDRESS_RANGE = 0x9000
mmio = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)

In [21]:
# PL_IS Config
ADDRESS_OFFSET = PL_IS #0x7000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

mmio.read(ADDRESS_OFFSET):  0x0


In [22]:
mmio.write(ADDRESS_OFFSET, 0x12345671)
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

mmio.read(ADDRESS_OFFSET):  0x1


In [23]:
mmio.write(ADDRESS_OFFSET, 0x12345673)
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

mmio.read(ADDRESS_OFFSET):  0x3


In [24]:
# PL_DMA Config
ADDRESS_OFFSET = PL_DMA # 0x8000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))
print("mmio.read(ADDRESS_OFFSET+0x20): ", hex(mmio.read(ADDRESS_OFFSET+0x20)))
print("mmio.read(ADDRESS_OFFSET+0x28): ", hex(mmio.read(ADDRESS_OFFSET+0x28)))
print("mmio.read(ADDRESS_OFFSET+0x30): ", hex(mmio.read(ADDRESS_OFFSET+0x30)))
print("mmio.read(ADDRESS_OFFSET+0x38): ", hex(mmio.read(ADDRESS_OFFSET+0x38)))
print("mmio.read(ADDRESS_OFFSET+0x3C): ", hex(mmio.read(ADDRESS_OFFSET+0x3C)))

mmio.read(ADDRESS_OFFSET):  0x4
mmio.read(ADDRESS_OFFSET+0x10):  0x0
mmio.read(ADDRESS_OFFSET+0x20):  0x0
mmio.read(ADDRESS_OFFSET+0x28):  0x0
mmio.read(ADDRESS_OFFSET+0x30):  0x0
mmio.read(ADDRESS_OFFSET+0x38):  0x0
mmio.read(ADDRESS_OFFSET+0x3C):  0x0


In [25]:
# PL_AS Config
ADDRESS_OFFSET = PL_AS # 0x6000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
mmio.write(ADDRESS_OFFSET, 0x12345676)
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

mmio.read(ADDRESS_OFFSET):  0x6
mmio.read(ADDRESS_OFFSET):  0x6


In [26]:
# PL_AA Config
ADDRESS_OFFSET = PL_AA # 0x2100
print("mmio.read(ADDRESS_OFFSET+0x00): ", hex(mmio.read(ADDRESS_OFFSET+0x00)))
print("mmio.read(ADDRESS_OFFSET+0x04): ", hex(mmio.read(ADDRESS_OFFSET+0x04))) 

mmio.read(ADDRESS_OFFSET+0x00):  0x0
mmio.read(ADDRESS_OFFSET+0x04):  0x0


In [27]:
mmio.write(ADDRESS_OFFSET+0x00, 0x11111111)
mmio.write(ADDRESS_OFFSET+0x04, 0x22222222)

In [28]:
# PL_AA_MB Mailbox
ADDRESS_OFFSET = PL_AA_MB # 0x2000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
print("mmio.read(ADDRESS_OFFSET+0x04): ", hex(mmio.read(ADDRESS_OFFSET+0x04)))
print("mmio.read(ADDRESS_OFFSET+0x08): ", hex(mmio.read(ADDRESS_OFFSET+0x08)))
print("mmio.read(ADDRESS_OFFSET+0x0C): ", hex(mmio.read(ADDRESS_OFFSET+0x0C)))
print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))
print("mmio.read(ADDRESS_OFFSET+0x14): ", hex(mmio.read(ADDRESS_OFFSET+0x14)))
print("mmio.read(ADDRESS_OFFSET+0x18): ", hex(mmio.read(ADDRESS_OFFSET+0x18)))
print("mmio.read(ADDRESS_OFFSET+0x1C): ", hex(mmio.read(ADDRESS_OFFSET+0x1C)))

mmio.read(ADDRESS_OFFSET):  0x0
mmio.read(ADDRESS_OFFSET+0x04):  0x0
mmio.read(ADDRESS_OFFSET+0x08):  0x0
mmio.read(ADDRESS_OFFSET+0x0C):  0x0
mmio.read(ADDRESS_OFFSET+0x10):  0x0
mmio.read(ADDRESS_OFFSET+0x14):  0x0
mmio.read(ADDRESS_OFFSET+0x18):  0x0
mmio.read(ADDRESS_OFFSET+0x1C):  0x0


In [29]:
mmio.write(ADDRESS_OFFSET, 0x11111112)
mmio.write(ADDRESS_OFFSET+0x04, 0x22222223)
mmio.write(ADDRESS_OFFSET+0x08, 0x33333334)
mmio.write(ADDRESS_OFFSET+0x0C, 0x44444445)
mmio.write(ADDRESS_OFFSET+0x10, 0x55555556)
mmio.write(ADDRESS_OFFSET+0x14, 0x66666667)
mmio.write(ADDRESS_OFFSET+0x18, 0x77777778)
mmio.write(ADDRESS_OFFSET+0x1C, 0x88888889)

In [30]:
# ====================================================================================== #
# ====================================================================================== #
# PL_Caravel Side Configuration
# ====================================================================================== #
# ====================================================================================== #

In [31]:
# Caravel-IS Config
ADDRESS_OFFSET = SOC_IS # 0x3000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

mmio.read(ADDRESS_OFFSET):  0x1


In [32]:
# Caravel-IS Config
ADDRESS_OFFSET = 0x3000
mmio.write(ADDRESS_OFFSET, 0x00000003)

In [33]:
# Caravel-AS Config
ADDRESS_OFFSET = SOC_AS # 0x4000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

mmio.read(ADDRESS_OFFSET):  0x6


In [34]:
# Caravel-AS Config
ADDRESS_OFFSET = SOC_AS # 0x4000
mmio.write(ADDRESS_OFFSET, 0x00000006)

In [35]:

# Caravel-CC Config
ADDRESS_OFFSET = SOC_CC # 0x5000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))


mmio.read(ADDRESS_OFFSET):  0x1f


In [36]:

# Caravel-CC Config
ADDRESS_OFFSET = SOC_CC # 0x5000
mmio.write(ADDRESS_OFFSET, 0x00000000)
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))


mmio.read(ADDRESS_OFFSET):  0x0


In [37]:
# Caravel-LA Config
ADDRESS_OFFSET = SOC_LA # 0x1000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

mmio.read(ADDRESS_OFFSET):  0x0


In [38]:
# Caravel-UP Config
ADDRESS_OFFSET = SOC_UP # 0x0000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET+0x04)))

mmio.read(ADDRESS_OFFSET):  0x280


In [39]:
# ====================================================================================== #
# FIR PL_UPDMA Configuration                                                             #
# ====================================================================================== #

In [40]:
# PL_IS Config
ADDRESS_OFFSET = PL_IS #0x7000
mmio.write(ADDRESS_OFFSET, 0x00000001)
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

mmio.read(ADDRESS_OFFSET):  0x1


In [41]:
# Caravel-IS Config
ADDRESS_OFFSET = 0x3000
mmio.write(ADDRESS_OFFSET, 0x00000003)
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

mmio.read(ADDRESS_OFFSET):  0x3


In [42]:
# Allocation memory m2s
mem_buf = allocate(shape=(64,), dtype=np.uint32)
print("mem_buf.device_address: ", hex(mem_buf.device_address))

MEM_BASE_ADDRESS = mem_buf.device_address
MEM_ADDRESS_RANGE = 0x0000_0400
data_mem = MMIO(MEM_BASE_ADDRESS, MEM_ADDRESS_RANGE)

MEM_ADDRESS_OFFSET = 0x00000000

# Allocation memory s2m
mem_buf2 = allocate(shape=(64,), dtype=np.uint32)
print("mem_buf2.device_address: ", hex(mem_buf2.device_address))

MEM_BASE_ADDRESS2 = mem_buf2.device_address
MEM_ADDRESS_RANGE2 = 0x0000_0400
data_mem2 = MMIO(MEM_BASE_ADDRESS2, MEM_ADDRESS_RANGE2)

MEM_ADDRESS_OFFSET2 = 0x00000000
print("data_mem2.read(MEM_ADDRESS_OFFSET2): ", hex(data_mem2.read(MEM_ADDRESS_OFFSET2)))



mem_buf.device_address:  0x16849000
mem_buf2.device_address:  0x1684a000
data_mem2.read(MEM_ADDRESS_OFFSET2):  0x0


In [43]:
print("Write input data to MEM...")
for idx in range(0, 64, 1):
  data_in = idx
  data_mem.write((MEM_ADDRESS_OFFSET+idx*4),data_in)
  print("No.", idx, ", mmio.read(", (MEM_ADDRESS_OFFSET+idx*4), ") = ", hex(data_mem.read(MEM_ADDRESS_OFFSET+idx*4)))

Write input data to MEM...
No. 0 , mmio.read( 0 ) =  0x0
No. 1 , mmio.read( 4 ) =  0x1
No. 2 , mmio.read( 8 ) =  0x2
No. 3 , mmio.read( 12 ) =  0x3
No. 4 , mmio.read( 16 ) =  0x4
No. 5 , mmio.read( 20 ) =  0x5
No. 6 , mmio.read( 24 ) =  0x6
No. 7 , mmio.read( 28 ) =  0x7
No. 8 , mmio.read( 32 ) =  0x8
No. 9 , mmio.read( 36 ) =  0x9
No. 10 , mmio.read( 40 ) =  0xa
No. 11 , mmio.read( 44 ) =  0xb
No. 12 , mmio.read( 48 ) =  0xc
No. 13 , mmio.read( 52 ) =  0xd
No. 14 , mmio.read( 56 ) =  0xe
No. 15 , mmio.read( 60 ) =  0xf
No. 16 , mmio.read( 64 ) =  0x10
No. 17 , mmio.read( 68 ) =  0x11
No. 18 , mmio.read( 72 ) =  0x12
No. 19 , mmio.read( 76 ) =  0x13
No. 20 , mmio.read( 80 ) =  0x14
No. 21 , mmio.read( 84 ) =  0x15
No. 22 , mmio.read( 88 ) =  0x16
No. 23 , mmio.read( 92 ) =  0x17
No. 24 , mmio.read( 96 ) =  0x18
No. 25 , mmio.read( 100 ) =  0x19
No. 26 , mmio.read( 104 ) =  0x1a
No. 27 , mmio.read( 108 ) =  0x1b
No. 28 , mmio.read( 112 ) =  0x1c
No. 29 , mmio.read( 116 ) =  0x1d
No. 30 

In [44]:
UPDMA_BASE_ADDRESS = 0x4002_0000
UPDMA_ADDRESS_RANGE = 0x0001_0000
updma = MMIO(UPDMA_BASE_ADDRESS, UPDMA_ADDRESS_RANGE)


UPDMA_ADDRESS_OFFSET = 0x0000 
#1
print("FpgaLocal_Write: PL_UPDMA, s2m exit clear...")
updma.write((UPDMA_ADDRESS_OFFSET+0x20),0x0000_0000)
print("mmio.read(ADDRESS_OFFSET): ", hex(updma.read(UPDMA_ADDRESS_OFFSET+0x20)))

#2
print("FpgaLocal_Write: PL_UPDMA, s2m disable to clear...")
updma.write((UPDMA_ADDRESS_OFFSET+0x30),0x0000_0000)
print("mmio.read(ADDRESS_OFFSET): ", hex(updma.read(UPDMA_ADDRESS_OFFSET+0x30)))

#3
print("FpgaLocal_Write: PL_UPDMA, m2s exit clear...")
updma.write((UPDMA_ADDRESS_OFFSET+0x78),0x0000_0000)
print("mmio.read(ADDRESS_OFFSET): ", hex(updma.read(UPDMA_ADDRESS_OFFSET+0x78)))

#4
print("FpgaLocal_Write: PL_UPDMA, m2s disable to clear...")
updma.write((UPDMA_ADDRESS_OFFSET+0x88),0x0000_0000)
print("mmio.read(ADDRESS_OFFSET): ", hex(updma.read(UPDMA_ADDRESS_OFFSET+0x88)))
'''
#5
print("FpgaLocal_Write: PL_UPDMA, s2m set buffer length....")
updma.write((UPDMA_ADDRESS_OFFSET+0x28),0x0000_E100)
print("mmio.read(ADDRESS_OFFSET): ", hex(updma.read(UPDMA_ADDRESS_OFFSET+0x28)))
'''

#6
print("FpgaLocal_Write: PL_UPDMA, s2m set buffer length....")
updma.write((UPDMA_ADDRESS_OFFSET+0x28),0x0000_0040)
print("mmio.read(ADDRESS_OFFSET): ", hex(updma.read(UPDMA_ADDRESS_OFFSET+0x28)))

#7
print("FpgaLocal_Write: PL_UPDMA, s2m set buffer low...")
#updma.write((UPDMA_ADDRESS_OFFSET+0x38),0x4508_0000)
updma.write((UPDMA_ADDRESS_OFFSET+0x38),mem_buf2.device_address)
print("mmio.read(ADDRESS_OFFSET): ", hex(updma.read(UPDMA_ADDRESS_OFFSET+0x38)))

#8
print("FpgaLocal_Write: PL_UPDMA, s2m set buffer high...")
updma.write((UPDMA_ADDRESS_OFFSET+0x3C),0x0000_0000)
print("mmio.read(ADDRESS_OFFSET): ", hex(updma.read(UPDMA_ADDRESS_OFFSET+0x3C)))

#9
print("FpgaLocal_Write: PL_UPDMA, set image width...")
updma.write((UPDMA_ADDRESS_OFFSET+0x54),0x0000_0040)
print("mmio.read(ADDRESS_OFFSET): ", hex(updma.read(UPDMA_ADDRESS_OFFSET+0x54)))

#10
print("FpgaLocal_Write: PL_UPDMA, m2s set buffer low...")
updma.write((UPDMA_ADDRESS_OFFSET+0x5C),mem_buf.device_address)
print("mmio.read(ADDRESS_OFFSET): ", hex(updma.read(UPDMA_ADDRESS_OFFSET+0x5C)))

#11
print("FpgaLocal_Write: PL_UPDMA, m2s set buffer high...")
updma.write((UPDMA_ADDRESS_OFFSET+0x60),0x0000_0000)
print("mmio.read(ADDRESS_OFFSET): ", hex(updma.read(UPDMA_ADDRESS_OFFSET+0x60)))

#12
print("FpgaLocal_Write: PL_UPDMA, m2s set buffer length...")
updma.write((UPDMA_ADDRESS_OFFSET+0x80),0x0000_0040)
print("mmio.read(ADDRESS_OFFSET): ", hex(updma.read(UPDMA_ADDRESS_OFFSET+0x80)))

FpgaLocal_Write: PL_UPDMA, s2m exit clear...
mmio.read(ADDRESS_OFFSET):  0x0
FpgaLocal_Write: PL_UPDMA, s2m disable to clear...
mmio.read(ADDRESS_OFFSET):  0x0
FpgaLocal_Write: PL_UPDMA, m2s exit clear...
mmio.read(ADDRESS_OFFSET):  0x0
FpgaLocal_Write: PL_UPDMA, m2s disable to clear...
mmio.read(ADDRESS_OFFSET):  0x0
FpgaLocal_Write: PL_UPDMA, s2m set buffer length....
mmio.read(ADDRESS_OFFSET):  0x40
FpgaLocal_Write: PL_UPDMA, s2m set buffer low...
mmio.read(ADDRESS_OFFSET):  0x1684a000
FpgaLocal_Write: PL_UPDMA, s2m set buffer high...
mmio.read(ADDRESS_OFFSET):  0x0
FpgaLocal_Write: PL_UPDMA, set image width...
mmio.read(ADDRESS_OFFSET):  0x40
FpgaLocal_Write: PL_UPDMA, m2s set buffer low...
mmio.read(ADDRESS_OFFSET):  0x16849000
FpgaLocal_Write: PL_UPDMA, m2s set buffer high...
mmio.read(ADDRESS_OFFSET):  0x0
FpgaLocal_Write: PL_UPDMA, m2s set buffer length...
mmio.read(ADDRESS_OFFSET):  0x40


In [45]:
# ====================================================================================== #
# FIR SOC_UP Configuration                                                             #
# ====================================================================================== #

In [46]:
# Caravel-CC Config
IP_BASE_ADDRESS = 0x60000000
ADDRESS_RANGE = 0x9000
mmio = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
ADDRESS_OFFSET = SOC_CC # 0x5000
mmio.write(ADDRESS_OFFSET, 0x00000001)
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
print(hex(IP_BASE_ADDRESS+ADDRESS_OFFSET))

mmio.read(ADDRESS_OFFSET):  0x1
0x60005000


In [47]:
# Caravel-UP Config
ADDRESS_OFFSET = SOC_UP # 0x0000
data_length = 0x0000_0040
#13
print("Fpga2Soc_Write: SOC_UP")
mmio.write((ADDRESS_OFFSET+0x10),data_length)
print("====> Write data_length, data_length = ", data_length)
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))

tap = [0x0000_0000, -0x0000_000A, -0x0000_0009, 0x0000_0017, 0x0000_0038, 
0x0000_003F, 0x0000_0038, 0x0000_0017, -0x0000_0009, -0x0000_000A, 0x0000_0000]

#14 - 24
#for x in range(len(tap)):
for idx in range(len(tap)):
  print("Fpga2Soc_Write: SOC_UP")
  print("====> Write tap[", idx, "], tap[", idx, "] = ", tap[idx])
  mmio.write((ADDRESS_OFFSET+0x20+idx*4),tap[idx])
  print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET+0x20+idx*4)))

Fpga2Soc_Write: SOC_UP
====> Write data_length, data_length =  64
mmio.read(ADDRESS_OFFSET):  0x40
Fpga2Soc_Write: SOC_UP
====> Write tap[ 0 ], tap[ 0 ] =  0
mmio.read(ADDRESS_OFFSET):  0x0
Fpga2Soc_Write: SOC_UP
====> Write tap[ 1 ], tap[ 1 ] =  -10
mmio.read(ADDRESS_OFFSET):  0xfffffff6
Fpga2Soc_Write: SOC_UP
====> Write tap[ 2 ], tap[ 2 ] =  -9
mmio.read(ADDRESS_OFFSET):  0xfffffff7
Fpga2Soc_Write: SOC_UP
====> Write tap[ 3 ], tap[ 3 ] =  23
mmio.read(ADDRESS_OFFSET):  0x17
Fpga2Soc_Write: SOC_UP
====> Write tap[ 4 ], tap[ 4 ] =  56
mmio.read(ADDRESS_OFFSET):  0x38
Fpga2Soc_Write: SOC_UP
====> Write tap[ 5 ], tap[ 5 ] =  63
mmio.read(ADDRESS_OFFSET):  0x3f
Fpga2Soc_Write: SOC_UP
====> Write tap[ 6 ], tap[ 6 ] =  56
mmio.read(ADDRESS_OFFSET):  0x38
Fpga2Soc_Write: SOC_UP
====> Write tap[ 7 ], tap[ 7 ] =  23
mmio.read(ADDRESS_OFFSET):  0x17
Fpga2Soc_Write: SOC_UP
====> Write tap[ 8 ], tap[ 8 ] =  -9
mmio.read(ADDRESS_OFFSET):  0xfffffff7
Fpga2Soc_Write: SOC_UP
====> Write tap[ 9 ], ta

In [48]:
# Caravel-CC Config
ADDRESS_OFFSET = SOC_CC # 0x5000
mmio.write(ADDRESS_OFFSET, 0x00000001)
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

#25
print("FpgaLocal_Write: SOC_UP, set ap_start...")
ADDRESS_OFFSET = SOC_UP
ap_start = 0x0000_0001
print(hex(IP_BASE_ADDRESS))
print(ADDRESS_OFFSET)
mmio.write((ADDRESS_OFFSET),ap_start)
print("====> Write data_length, ap_start = ", ap_start)
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))



mmio.read(ADDRESS_OFFSET):  0x1
FpgaLocal_Write: SOC_UP, set ap_start...
0x60000000
0
====> Write data_length, ap_start =  1
mmio.read(ADDRESS_OFFSET):  0x1


In [49]:
UPDMA_BASE_ADDRESS = 0x4002_0000
UPDMA_ADDRESS_RANGE = 0x0001_0000
updma = MMIO(UPDMA_BASE_ADDRESS, UPDMA_ADDRESS_RANGE)


UPDMA_ADDRESS_OFFSET = 0x0000 

#26
print("FpgaLocal_Write: PL_UPDMA, set ap_start...")
print(UPDMA_ADDRESS_OFFSET)
updma.write((UPDMA_ADDRESS_OFFSET),0x0000_0001)
print("updma.read(UPDMA_ADDRESS_OFFSET): ", hex(updma.read(UPDMA_ADDRESS_OFFSET)))
print("UPDMA_ADDRESS_OFFSET = ",hex(UPDMA_BASE_ADDRESS + UPDMA_ADDRESS_OFFSET))

FpgaLocal_Write: PL_UPDMA, set ap_start...
0
updma.read(UPDMA_ADDRESS_OFFSET):  0xe
UPDMA_ADDRESS_OFFSET =  0x40020000


In [50]:
print(hex(UPDMA_BASE_ADDRESS+UPDMA_ADDRESS_OFFSET+0x10))

while True:
    if updma.read(UPDMA_ADDRESS_OFFSET+0x10) == 0x01:
        break
print("updma.read(ADDRESS_OFFSET+0x10): ", hex(updma.read(UPDMA_ADDRESS_OFFSET+0x10)))

0x40020010
updma.read(ADDRESS_OFFSET+0x10):  0x1


In [51]:
print("Print output data from MEM...")
MEM_ADDRESS_OFFSET2 = 0x0000_0000
for idx in range(0, 64, 1):
  print("No.", idx, ", mmio.read(", (MEM_ADDRESS_OFFSET2+idx*4), ") = ", hex(data_mem2.read(MEM_ADDRESS_OFFSET2+idx*4)))

Print output data from MEM...
No. 0 , mmio.read( 0 ) =  0x0
No. 1 , mmio.read( 4 ) =  0x0
No. 2 , mmio.read( 8 ) =  0xfffffff6
No. 3 , mmio.read( 12 ) =  0xffffffe3
No. 4 , mmio.read( 16 ) =  0xffffffe7
No. 5 , mmio.read( 20 ) =  0x23
No. 6 , mmio.read( 24 ) =  0x9e
No. 7 , mmio.read( 28 ) =  0x151
No. 8 , mmio.read( 32 ) =  0x21b
No. 9 , mmio.read( 36 ) =  0x2dc
No. 10 , mmio.read( 40 ) =  0x393
No. 11 , mmio.read( 44 ) =  0x44a
No. 12 , mmio.read( 48 ) =  0x501
No. 13 , mmio.read( 52 ) =  0x5b8
No. 14 , mmio.read( 56 ) =  0x66f
No. 15 , mmio.read( 60 ) =  0x726
No. 16 , mmio.read( 64 ) =  0x7dd
No. 17 , mmio.read( 68 ) =  0x894
No. 18 , mmio.read( 72 ) =  0x94b
No. 19 , mmio.read( 76 ) =  0xa02
No. 20 , mmio.read( 80 ) =  0xab9
No. 21 , mmio.read( 84 ) =  0xb70
No. 22 , mmio.read( 88 ) =  0xc27
No. 23 , mmio.read( 92 ) =  0xcde
No. 24 , mmio.read( 96 ) =  0xd95
No. 25 , mmio.read( 100 ) =  0xe4c
No. 26 , mmio.read( 104 ) =  0xf03
No. 27 , mmio.read( 108 ) =  0xfba
No. 28 , mmio.read(

In [52]:
# ====================================================================================== #
# ====================================================================================== #
# LADMA Verification
# ====================================================================================== #
# ====================================================================================== #

In [53]:
# Allocation memory
ladma_buf = allocate(shape=(1024,), dtype=np.uint32)
print("ladma_buf.device_address: ", hex(ladma_buf.device_address))

IP_BASE_ADDRESS = ladma_buf.device_address
ADDRESS_RANGE = 0x1000
buf_mmio = MMIO(IP_BASE_ADDRESS, ADDRESS_RANGE)
# 0x00 : Control signals
#        bit 0 - ap_start (R/W/COH)
#        bit 1 - ap_done (R/COR)
#        bit 2 - ap_idle (R)
#        bit 3 - ap_ready (R/COR)
# 0x10 : Buffer transfer done status register
#        bit 0 - buffer transfer done status (R)
# 0x20 : Buffer transfer done status clear register
#        bit 0 - clear buffer transfer done status (R/W)
# 0x28 : Buffer Length
#        bit 31~0 - set buffer length (must 1024)
# 0x30 : Triggered condition 
#        bit 23~0 - set triggered condidtion (R/W)
#        others  - reserved
# 0x34 : Buffer Lower base address 
#        bit 31~0 - (R/W)
# 0x38 : Buffer High base address
#        bit 31~0 - (R/W)
# ladma Configuration
ADDRESS_OFFSET = PL_DMA # 0x8000
# exit clear operation
mmio.write(ADDRESS_OFFSET + 0x20, 0x00000000)
# set buffer length
mmio.write(ADDRESS_OFFSET + 0x28, 0x00000400)
# set trigger condition
mmio.write(ADDRESS_OFFSET + 0x30, 0x00000000)
# set buffer low
mmio.write(ADDRESS_OFFSET + 0x38, ladma_buf.device_address)
# set buffer high
mmio.write(ADDRESS_OFFSET + 0x3C, 0x00000000)

ladma_buf.device_address:  0x1684b000


In [54]:
ADDRESS_OFFSET = PL_DMA # 0x8000
print("mmio.read(ADDRESS_OFFSET+0x00): ", hex(mmio.read(ADDRESS_OFFSET+0x00)))
print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))
print("mmio.read(ADDRESS_OFFSET+0x20): ", hex(mmio.read(ADDRESS_OFFSET+0x20)))
print("mmio.read(ADDRESS_OFFSET+0x28): ", hex(mmio.read(ADDRESS_OFFSET+0x28)))
print("mmio.read(ADDRESS_OFFSET+0x30): ", hex(mmio.read(ADDRESS_OFFSET+0x30)))
print("mmio.read(ADDRESS_OFFSET+0x38): ", hex(mmio.read(ADDRESS_OFFSET+0x38)))
print("mmio.read(ADDRESS_OFFSET+0x3C): ", hex(mmio.read(ADDRESS_OFFSET+0x3C)))

mmio.read(ADDRESS_OFFSET+0x00):  0x4
mmio.read(ADDRESS_OFFSET+0x10):  0x0
mmio.read(ADDRESS_OFFSET+0x20):  0x0
mmio.read(ADDRESS_OFFSET+0x28):  0x400
mmio.read(ADDRESS_OFFSET+0x30):  0x0
mmio.read(ADDRESS_OFFSET+0x38):  0x1684b000
mmio.read(ADDRESS_OFFSET+0x3C):  0x0


In [55]:
# ladma Configuration
ADDRESS_OFFSET = PL_DMA # 0x8000
# set ap_start
mmio.write(ADDRESS_OFFSET + 0x00, 0x00000001)

In [56]:
# enable la 0xFFFFFF
ADDRESS_OFFSET = SOC_LA # 0x1000
mmio.write(ADDRESS_OFFSET, 0x00FFFFFF)
#print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
# select target UP
ADDRESS_OFFSET = SOC_CC # 0x5000
mmio.write(ADDRESS_OFFSET, 0x00000003)
#print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

In [57]:
# ladma Configuration
ADDRESS_OFFSET = PL_DMA # 0x8000
while True:
    if mmio.read(ADDRESS_OFFSET+0x10) == 0x01:
        break
print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))

# disable la 0x000000
ADDRESS_OFFSET = SOC_LA # 0x1000
mmio.write(ADDRESS_OFFSET, 0x00000000)
# select fake UP
ADDRESS_OFFSET = SOC_CC # 0x5000
mmio.write(ADDRESS_OFFSET, 0x00000000) 

ADDRESS_OFFSET = PL_DMA # 0x8000
# clear buffer transfer done operation
mmio.write(ADDRESS_OFFSET + 0x20, 0x00000001)
# set ap_start
mmio.write(ADDRESS_OFFSET + 0x00, 0x00000001)
while True:
    if mmio.read(ADDRESS_OFFSET+0x10) != 0x01:
        break
print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))      

mmio.read(ADDRESS_OFFSET+0x10):  0x1
mmio.read(ADDRESS_OFFSET+0x10):  0x0


In [58]:
ADDRESS_OFFSET = PL_DMA # 0x8000
print("mmio.read(ADDRESS_OFFSET+0x00): ", hex(mmio.read(ADDRESS_OFFSET+0x00)))
print("mmio.read(ADDRESS_OFFSET+0x10): ", hex(mmio.read(ADDRESS_OFFSET+0x10)))
print("mmio.read(ADDRESS_OFFSET+0x20): ", hex(mmio.read(ADDRESS_OFFSET+0x20)))
print("mmio.read(ADDRESS_OFFSET+0x28): ", hex(mmio.read(ADDRESS_OFFSET+0x28)))
print("mmio.read(ADDRESS_OFFSET+0x30): ", hex(mmio.read(ADDRESS_OFFSET+0x30)))
print("mmio.read(ADDRESS_OFFSET+0x38): ", hex(mmio.read(ADDRESS_OFFSET+0x38)))
print("mmio.read(ADDRESS_OFFSET+0x3C): ", hex(mmio.read(ADDRESS_OFFSET+0x3C)))

mmio.read(ADDRESS_OFFSET+0x00):  0xe
mmio.read(ADDRESS_OFFSET+0x10):  0x0
mmio.read(ADDRESS_OFFSET+0x20):  0x1
mmio.read(ADDRESS_OFFSET+0x28):  0x400
mmio.read(ADDRESS_OFFSET+0x30):  0x0
mmio.read(ADDRESS_OFFSET+0x38):  0x1684b000
mmio.read(ADDRESS_OFFSET+0x3C):  0x0


In [59]:
#dump la log to file
file = open("simulate.log", "w")
for i in range(0,0xFFF,4):
    file.write('{:08x}'.format(buf_mmio.read(i))+"\n")
file.close()

In [60]:
# ladma Configuration
ADDRESS_OFFSET = PL_DMA # 0x8000
# exit clear operation
mmio.write(ADDRESS_OFFSET + 0x20, 0x00000000)
# set ap_start again
mmio.write(ADDRESS_OFFSET + 0x00, 0x00000001)

# Confirming SOC_CC & SOC_LA Reading is still work
ADDRESS_OFFSET = SOC_CC # 0x5000
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))
print("mmio.read(ADDRESS_OFFSET): ", hex(mmio.read(ADDRESS_OFFSET)))

mmio.read(ADDRESS_OFFSET):  0x0
mmio.read(ADDRESS_OFFSET):  0x0


In [61]:
# Translate simulate.log to output_new.vcd, which can be open by gtakwave
%run read_la_data.py